# TensorBoard - Ladění hyperparametrů ⚒️
Jak už asi víte z jiných předmětů, při vytváření ML modelů hrají velkou roli hyperparametry. Jejich správná volba se může výrazně projevit v hodnotě metriky, kterou zkoumáme (např. přesnost). Právě proto je hledání nejlepších hyperparametrů 🔎 velmi častým úkolem a někdy nezbývá nic jiného, než zkusit všechny (nebo nějaké rozumně zvolené) možnosti.

V tomto notebooku si ukážeme, jak nám s tímto úkolem může pomoci TensorBoard právě díky vizualizacím, které nabízí. K vysvětlení budeme používat stejný modelový problém jako v předchozích noteboocích – neuronovou síť na datasetu MNIST ✍️.

In [ ]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

In [ ]:
%load_ext tensorboard

# removes logs from previous runs
!rm -rf ./logs/ 

In [ ]:
# this code is copied from first notebook
mnist = tf.keras.datasets.mnist

(X_train, y_train),(X_test, y_test) = mnist.load_data()

X_train = X_train / 255.0
X_test = X_test / 255.0

⚙️ Budeme ladit tři hyperparametry:
* dropout - hodnoty v intervalu <0.1, 0.25> (= regularizace pomocí vynulování některých neuronů)
* optimizer - Adam nebo sgd (= metoda pro update vah)
* počet neuronů v první dense vrstvě - 16 nebo 32 (= plně propojená, fully connected)

Tuto konfiguraci hyperparametrů zalogujeme do TensorBoardu, později budeme díky tomu moci filtrovat určité hodnoty hyperparamterů a metriky.

In [ ]:
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.25))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))

ACCURACY = 'accuracy'
log_dir = 'logs/hparam_tuning'

with tf.summary.create_file_writer(log_dir).as_default():
    # logs hyperparameters configuration
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        # specify which metrics should be displayed
        metrics=[hp.Metric(ACCURACY, display_name='Accuracy')],
    )

Jak to zatím vypadá 👀 ?

In [ ]:
%tensorboard --logdir logs/hparam_tuning

Zatím nám v TensorBoardu chybí to hlavní – přesnosti vzhledem ke konkrétním kombinacím hyperparametrů. Pojďme je změřit a zalogovat. S měřením 📐 nám pomůže tato funkce.

Kód je velmi podobný tomu, co jsme viděli v předchozích noteboocích. Například při vytváření Keras modelu, kde v poli zadáváme jednotlivé vrstvy, ze kterých se má model skládat, jsme v předchozích noteboocích měli tuto definici dense vrstvy:

```python
tf.keras.layers.Dense(512, activation='relu')
````
nyní je definice podobná, ale počet neuronů zadáváme parametrem:

```python
tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation='relu')
````

Podobné změny nastaly i při kompilaci modelu. V závěru funkce model natrénujeme 🏋️  na trénovacích datech (pouze v jedné epoše, aby výpočet netrval dlouho), změříme jeho přesnost na validačních datech a tuto přesnost vrátíme.

In [ ]:
# creates model with specified hyperparameters
# returns its accuracy on validation data
def compute_accuracy(hparams):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation='relu'),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(10, activation='softmax'),
    ])
    
    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'],
    )

    # 1 epoch for demo purposes, you will need a lot more in practice
    model.fit(X_train, y_train, epochs=1) 
    _, accuracy = model.evaluate(X_test, y_test)
    return accuracy

Pomocí předchozí funkce `compute_accuracy` umíme změřit přesnost modelu vytvořeného s nějakou kombinací hyperparametrů. Funkce níže se postará o zalogování do TensorBoardu. Nejdříve zaloguje hyperparametry, poté vypočítá přesnost pomocí funkce `compute_accuracy` a zaloguje ji.

In [ ]:
def log_accuracy(log_dir, hparams):
    with tf.summary.create_file_writer(log_dir).as_default():
        # logs hyperparameter values used in this run
        hp.hparams(hparams)
        # trains model with specified hyperparameters and computes accuracy
        accuracy = compute_accuracy(hparams)
        # logs the accuracy
        tf.summary.scalar(ACCURACY, accuracy, step=1)

A už zbývá jen jedna věc – zavolat předchozí funkci `log_accuracy` pro každou kombinaci hyperparametrů. Kombinace hyperparametrů vytvoříme pomocí tří vnořených for cyklů – jeden pro každý hyperparametr. Dovolili jsme si udělat menší zjednodušení a vyzkoušíme jen minimální a maximální hodnotu pro dropout hyperparametr, jehož interval je <0.1,0.25>.

In [ ]:
run_num = 0

# 3 nested fors to get every combination of hyperparameters
for num_units in HP_NUM_UNITS.domain.values:
    for dropout in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout,
                HP_OPTIMIZER: optimizer,
            }
            
            log_accuracy('logs/hparam_tuning/{}'.format(run_num), hparams)
            run_num += 1

## 📈 Výsledky

Podívejme se na výsledky. Když se otevře TensorBoard, je třeba v horním navigačním panelu kliknout na záložku HParams.

V levém panelu je pak možné filtrovat, které hyperparametry či hodnoty hyperparametrů a metrik se mají zobrazovat. Můžeme si vybrat i to, jak se mají data seřadit apod.

HParams dashboard obsahuje tři taby:
* TableView
     * tabulka, kde je pro každý běh uvedena výsledná metrika a hyperparametry, které byly použity
* Parallel Coordinates View
     * graf paralelních souřadnic
     * na jednotlivých osách můžeme filtrovat, které hodnoty nás zajímají
* Scatter Plot View
     * skupina bodových grafů pro každý hyperparametr a metriku
     * když označíte nějaký region v jednom grafu, vyfiltrují se tím data i v ostatních grafech

In [ ]:
%tensorboard --logdir logs/hparam_tuning

Chcete-li si prohlédnout HParams dashboard detailněji, ve složce hparams_demo máte více dat, než jsme si vygenerovali v tomto notebooku 🙂

# A to je dnes vše! 🎉